# **Measure Organelle Morphology**

***Prior to this notebook, you should have already run through [2.0_quantification_setup](2.0_quantification_setup.ipynb).***

In notebooks 2.1 through 2.4, we will go over the implementation of `infer-subc` quantification methods (explained in detail in the `method_...` notebooks) to assess the morphology, interactions, and distribution of organelles at the single-cell level. 

### 📍 **Purpose**

This notebook measures the amount, size, and shape of organelles. It includes an option to batch process the morphology measurements for `mulitple organelles` across `multiple cells`.

### 🍃 **Biological Relevance**
Measurements of organelle morphology -- amount, size, and shape -- are included as part of the organelle signature analysis. These metrics can provide information about the physiology of a cell and its constituent organelles. 

Organelle amounts have been demonstrated to differ between cell types. The cytoplasms of some specialized cell types, like adipocytes, are composed almost entirely by a single large lipid droplet for fat storage[[1]](https://doi.org/10.3390/biom11121906), while other cell types, like muscle cells, have large and elaborate mitochdondrial networks for effective metabolite diffusion during muscle contraction[[2]](https://doi.org/10.1038/nature14614).

Additionally, different organelle morphologies are important to their function. For example, fission and fusion maintain mitochondrial homeostasis by modulating the size and connectedness of the mitochondrial network. A recent study demonstrated that asymmetric fission events resulted in morphologically distinct daughter mitochondria with different fates; the larger mitochondria continued to grow and divide, while the smaller, spherical fragments containing high reactive oxygen species (ROS) were destined for autophagic degradation[[3]](https://doi.org/10.1038/s41586-021-03510-6).

### 📐 **Regionprops Morphology Measurements** 

*You can learn more about the implementation of regionprops within infer-subc in the [method_morphology](method_morphology.ipynb) notebook.*

The following morphological measurements are included:
- `label`: the unique ID number for the object being measured
- `centroid`: centroid coordinate tuple (row, col, Z)
- `bbox`: bounding box coordinates (min_row, min_col, max_row, max_col); pixels/voxels belonging to the bounding box are in the half-open interval [min_row; max_row) and [min_col; max_col).
- `area`: (or `volume` for 3D z-stack images) area of the region i.e. number of pixels of the region scaled by pixel-area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `equivalent_diameter`: the diameter of a circle with the same area as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `extent`: ratio of pixels/voxels in the region to pixels/voxels in the total bounding box. Computed as area / (rows * cols)
- `euler_number`: Euler characteristic of the set of non-zero pixels. Computed as number of connected components subtracted by number of holes (input.ndim connectivity). In 3D, number of connected components plus number of holes subtracted by number of tunnels.
- `solidity`: ratio of pixels/voxels in the region to pixels/voxels of the convex hull image.
- `axis_major_length`: the length of the major axis of the ellipse that has the same normalized second central moments as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `surface_area`: the surface area of the region. For 3D, surface area of a 2D surface mesh of the region (skimage.measure.marching_cubes) using skimage.measure.mesh_surface_area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `SA_to_volume`: surface area / area (or volume); this metric has the option to be converted into "real world" units using the scale from the metadata.

The following measures of the intensity images are also included:
- `min_intensity`: value with the least intensity in the region.
- `max_intensity`: value with the greatest intensity in the region.
- `mean_intensity`: value with the mean intensity in the region.
- `standard_deviation_intensity`: the standard deviation of the intensity in the region.

These measurements and definitions are derived from the [`skimage.measure.regionprops()`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) function. More in depth information about each measurement can be found there.

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`2.0_quantification_setup`](2.0_quantification_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

In [74]:
from typing import List, Union
from pathlib import Path
import os
import time

from infer_subc.core.img import *

import numpy as np
import pandas as pd
import napari
from napari.utils.notebook_display import nbscreenshot

from infer_subc.utils.stats import get_morphology_metrics
from infer_subc.utils.batch import list_image_files, find_segmentation_tiff_files
from infer_subc.core.file_io import read_czi_image, read_tiff_image

pd.set_option('display.max_columns', None)

#### &#x1F3C3; **Run code; no user input required**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `raw_img_type`, `data_root_path`, `raw_data_path`, `seg_data_path`, and `quant_data_path`.

In [2]:
#### USER INPUT REQUIRED ###
raw_img_type = ".czi"
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
raw_data_path = data_root_path / "raw_single"
seg_data_path = data_root_path / "out_single"
quant_data_path = data_root_path / "quant_single"

#### &#x1F3C3; **Run code; no user input required**

In [3]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(quant_data_path):
    Path.mkdir(quant_data_path)
    print(f"making {quant_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
raw_img_file_list = list_image_files(raw_data_path,raw_img_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [4]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [5]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = raw_img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']

print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

Metadata information
File path: C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi
Channel 0 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Nuclei_Jan22
Channel 1 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Lyso+405_Jan22
Channel 2 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Mito+405_Jan22
Channel 3 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Golgi+405_Jan22
Channel 4 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Peroxy+405_Jan22
Channel 5 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: ER+405_Jan22
Channel 6 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: BODIPY+405low_Jan22
Channel 7 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Residuals
Scale (ZYX): (0.3891184878080979, 0.07987165184837317, 0.07987165184837318)
Channel axis: 0


#### &#x1F6D1; &#x270D; **User Input Required:**

Specify the following information about the segmentation files: - `org_file_names`, `org_channels_ordered`, `regions_file_names`, and `suffix_separator`.

In [6]:
#### USER INPUT REQUIRED ###
org_file_names = ["lyso", "mito", "golgi", "perox", "ER", "LD"]
org_channels_ordered = [1, 2, 3, 4, 5, 6]
regions_file_names = ["cell", "nuc"]
suffix_separator = "-"

#### &#x1F3C3; **Run code; no user input required**

In [7]:
# find file paths for segmentations
all_suffixes = org_file_names + regions_file_names
filez = find_segmentation_tiff_files(file_path, all_suffixes, seg_data_path, suffix_separator)

# read the segmentation and masks/regions files into memory
organelles = [read_tiff_image(filez[org]) for org in org_file_names]
regions = [] 
for m in regions_file_names:
    mfile = read_tiff_image(filez[m])
    regions.append(mfile)

# match the intensity channels to the segmentation files
intensities = [img_data[ch] for ch in org_channels_ordered]

# open viewer and add images
viewer = napari.Viewer()
for r, reg in enumerate(regions_file_names):
    viewer.add_image(regions[r],
                     scale=scale,
                     name=f"{reg} mask")

# colors = ["red", "bop orange", "yellow", "green", "blue", "cyan", "magenta", "bop purple"]
for o, org in enumerate(org_file_names):
    viewer.add_image(intensities[o],
                     scale=scale,
                     name=f"{org} intensity channel")
    viewer.add_labels(organelles[o],
                      scale=scale,
                      name=f"{org} segmentation")
viewer.grid.enabled = True
viewer.reset_view()

print("The following matching files were found and can now be viewed in Napari:")
filez

03-Dec-24 13:33:27 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1086x661+1532+199 (frame: 1102x700+1524+168) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY2". Resulting geometry: 867x527+1531+192 (frame: 883x566+1523+161) margins: 8, 31, 8, 8 minimum size: 385x513 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=401,552 maxtrack=0,0)


The following matching files were found and can now be viewed in Napari:


{'raw': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw_single/a24hrs_Ctrl_14_Unmixing.czi'),
 'lyso': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-lyso.tiff'),
 'mito': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-mito.tiff'),
 'golgi': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-golgi.tiff'),
 'perox': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-perox.tiff'),
 'ER': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-ER.tiff'),
 'LD': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-LD.tiff'),
 'cell': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-cell.tiff'),
 'nuc': WindowsPath('C:/Use

------

## **EXPLANATION OF STEPS**

### 🦠 **Processing One or More Organelles from <ins>ONE CELL</ins>**

#### **`STEP 1` - Select mask to be used**

&#x1F453; **FYI:** To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask to the segmentation file. This will exclude any objects outside of the mask area from the analysis.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify which organelle you would like to examine in this analysis and which segmentation file should be used as the mask:
- `mask_name`: the suffix of the mask you would like to use; it should match one of the names included in 

In [8]:
#### USER INPUT REQUIRED ###
mask_name = "cell"

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** The mask file is selected from the list of regions and added to Napari for visual inspection if desirer.

In [60]:
# select the mask from the region list
mask = regions[regions_file_names.index(mask_name)]

# add mask to napari for visual inspection
viewer.layers.clear()
viewer.add_image(img_data, scale=scale, name="Intensity Image")
viewer.add_labels(mask, scale=scale, name="Mask")
viewer.grid.enabled = False
viewer.reset_view()

#### **`STEP 2` - Loop through the list of organelles to quantify the morphology of each**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** The block of code below loops through the list of organelles to:
1) Select the intensity image that organelle segmentation was derived from
2) Select the organelle segmentation image
3) Assure the segmentation files are formatted corrected (e.g., the ER should only include on object)

> ***IMPORTANT**: The solidity measurement may cause an error for objects that are very small. It depends on the convex hull measurement which is computed as 0 when the object volume is very small; the solidity value is them output in the table as `inf` (infinity). The following error message will be included:*
> ```python
> UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
> ```

In [71]:
# empty list to collect a morphology data for each organelle
org_tabs = []

# loop through the list of organelles and run the get_morphology_metrics function
for j, target in enumerate(org_file_names):
    # select intensity image
    org_img = intensities[j]  
    
    # select segmentation and if ER, ensure it is only one object
    if target == 'ER':
        org_obj = (organelles[j] > 0).astype(np.uint16)  
    else:
        org_obj = organelles[j]
    
    # run get_morphology_metrics function to output a table of measurements
    org_metrics = get_morphology_metrics(segmentation_img=org_obj, 
                                        seg_name=target,
                                        intensity_img=org_img, 
                                        mask=mask,
                                        scale=scale)

    # add table to list above
    org_tabs.append(org_metrics)

# print each table separately
for i, org in enumerate(org_file_names):
    print(f"{org} morphology metrics table:")
    display(org_tabs[i])

c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6214 qhull input error: not enough points(2) to construct initial simplex (need 4)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 328039909  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:629: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    

lyso morphology metrics table:


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,lyso,1,"(0.3891, 0.0799, 0.0799)",0.075313,21.835879,15.873847,0,271,197,2,277,202,1.001157,0.076954,13.009875,0.527728,0.516667,1,0.837838,0.745434,0.0,6740.0,2662.774194,1873.980096
1,lyso,2,"(0.3891, 0.0799, 0.0799)",0.000000,21.685153,20.527015,0,271,257,1,273,258,0.152282,0.004965,30.672726,0.211657,1.000000,1,inf,0.178598,992.0,2471.0,1731.500000,739.500000
2,lyso,3,"(0.3891, 0.0799, 0.0799)",0.000000,22.265759,19.359660,0,276,239,1,283,246,0.806408,0.064542,12.494366,0.497677,0.530612,1,inf,0.860433,174.0,4247.0,1975.115385,968.798992
3,lyso,4,"(0.3891, 0.0799, 0.0799)",3.172964,23.723579,17.715020,0,226,167,16,361,283,389.806620,65.135009,5.984595,4.991970,0.104721,-17,0.233281,10.902797,0.0,24052.0,5743.695110,3725.788256
4,lyso,5,"(0.3891, 0.0799, 0.0799)",0.148781,24.490058,25.988826,0,305,323,2,310,329,1.046691,0.084401,12.401451,0.544230,0.566667,1,0.918919,0.845667,0.0,5711.0,2080.500000,1527.815824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,lyso,494,"(0.3891, 0.0799, 0.0799)",6.141305,17.929449,17.722825,15,222,219,17,228,226,1.180759,0.114189,10.340370,0.601925,0.547619,1,0.836364,0.740014,208.0,7240.0,2510.260870,1806.076584
367,lyso,495,"(0.3891, 0.0799, 0.0799)",5.836777,22.124448,23.382426,15,276,292,16,279,294,0.494978,0.009929,49.849288,0.266671,0.666667,1,inf,0.252576,315.0,2231.0,1341.500000,783.406823
368,lyso,496,"(0.3891, 0.0799, 0.0799)",5.836777,25.188615,25.159570,15,314,313,16,318,318,0.914082,0.027306,33.475374,0.373612,0.550000,1,inf,0.443638,0.0,3784.0,1492.090909,955.067580
369,lyso,497,"(0.3891, 0.0799, 0.0799)",5.836777,28.807042,25.292690,15,360,316,16,362,318,0.399771,0.007447,53.681343,0.242287,0.750000,1,inf,0.206228,0.0,1247.0,801.000000,567.609608


mito morphology metrics table:


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,mito,1,"(0.3891, 0.0799, 0.0799)",3.250337,23.730830,21.955442,0,264,211,15,343,338,113.813081,13.913706,8.179926,2.984109,0.037244,0,0.148402,13.458692,0.0,42566.0,14153.676896,6789.082433
1,mito,3,"(0.3891, 0.0799, 0.0799)",0.625799,24.706690,23.437801,0,305,287,4,316,301,4.125601,0.240790,17.133588,0.771875,0.157468,1,0.692857,1.436974,1538.0,19544.0,9617.793814,3279.118403
2,mito,4,"(0.3891, 0.0799, 0.0799)",0.832400,26.550303,18.437332,0,315,224,5,349,238,12.046763,1.355376,8.888132,1.373052,0.229412,1,0.588362,3.566324,0.0,34053.0,13149.695971,5979.664482
3,mito,7,"(0.3891, 0.0799, 0.0799)",1.809289,26.442872,19.974654,0,319,239,10,345,272,18.580541,2.147253,8.653166,1.600645,0.100816,1,0.339615,4.291476,0.0,35846.0,13743.396532,6476.447011
4,mito,8,"(0.3891, 0.0799, 0.0799)",2.441321,30.106720,15.356437,0,331,151,12,413,229,112.895798,16.046065,7.035731,3.129368,0.084219,-5,0.247891,7.406881,0.0,43601.0,15640.577970,6499.348811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,mito,86,"(0.3891, 0.0799, 0.0799)",5.186171,27.350609,22.064810,12,333,262,16,360,296,11.628860,0.930890,12.492193,1.211435,0.102124,1,0.509511,4.046913,0.0,22519.0,9168.416000,3766.492186
64,mito,87,"(0.3891, 0.0799, 0.0799)",5.668451,20.822719,17.434905,13,253,214,17,269,224,4.906275,0.441863,11.103622,0.944995,0.278125,1,0.652015,1.789801,2265.0,23577.0,11145.887640,4484.971203
65,mito,88,"(0.3891, 0.0799, 0.0799)",5.225305,31.284965,24.136453,13,390,299,15,394,306,1.936242,0.104260,18.571336,0.583946,0.750000,1,0.954545,0.866383,3496.0,12744.0,7999.714286,2264.419639
66,mito,89,"(0.3891, 0.0799, 0.0799)",5.514365,19.999862,16.850636,14,247,208,16,255,214,1.942056,0.086883,22.352524,0.549514,0.364583,1,0.813953,0.767099,0.0,16982.0,8294.314286,4177.945338


...

,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,perox,1,"(0.3891, 0.0799, 0.0799)",0.622590,34.355460,18.972180,1,429,236,3,433,240,1.121680,0.037236,30.123863,0.414305,0.468750,1,0.937500,0.879628,442.0,20765.0,8072.200000,5641.951261
1,perox,3,"(0.3891, 0.0799, 0.0799)",0.965590,23.369854,21.216277,2,291,264,4,296,268,1.461175,0.067024,21.800749,0.503977,0.675000,1,1.000000,0.873170,1841.0,25484.0,9061.925926,5685.575992
2,perox,4,"(0.3891, 0.0799, 0.0799)",1.198485,23.386420,24.239449,2,291,302,5,295,306,1.519463,0.062059,24.484029,0.491213,0.520833,1,0.806452,1.103160,0.0,20059.0,9041.880000,5572.241311
3,perox,5,"(0.3891, 0.0799, 0.0799)",1.052909,38.357186,21.828453,2,478,271,4,483,276,1.307596,0.042200,30.985421,0.431956,0.340000,1,0.739130,0.902121,0.0,12437.0,7640.470588,3372.797689
4,perox,10,"(0.3891, 0.0799, 0.0799)",1.167355,24.311702,12.988359,3,303,161,4,307,165,0.920559,0.032271,28.526013,0.395006,0.812500,1,inf,0.385654,1473.0,14763.0,8172.461538,4853.273100
5,perox,11,"(0.3891, 0.0799, 0.0799)",1.167355,27.316105,15.814587,3,341,197,4,344,200,0.736331,0.022341,32.958181,0.349438,1.000000,1,inf,0.291650,0.0,16451.0,9146.555556,5080.729237
6,perox,12,"(0.3891, 0.0799, 0.0799)",1.450351,32.268147,15.426120,3,402,191,5,407,196,1.402827,0.054612,25.687047,0.470721,0.440000,1,0.846154,0.820496,0.0,21069.0,9346.863636,5680.059301
7,perox,13,"(0.3891, 0.0799, 0.0799)",1.310715,32.755785,17.533929,3,409,217,5,413,223,1.495411,0.047165,31.705872,0.448271,0.395833,1,0.703704,0.936137,3440.0,17384.0,7683.105263,3726.200321
8,perox,14,"(0.3891, 0.0799, 0.0799)",1.211826,33.883837,20.536143,3,419,255,5,429,260,2.100261,0.086883,24.173415,0.549514,0.350000,1,0.744681,1.018209,110.0,18969.0,8361.314286,4185.351610
9,perox,15,"(0.3891, 0.0799, 0.0799)",1.369697,36.801662,20.146825,3,459,250,5,463,255,1.423716,0.062059,22.941210,0.491213,0.625000,1,0.961538,0.880769,0.0,18847.0,7018.560000,4236.233828


ER morphology metrics table:


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,ER,1,"(0.3891, 0.0799, 0.0799)",2.602363,28.710334,22.73029,0,123,102,17,632,566,3622.111476,893.570207,4.053528,11.950242,0.089655,-124,0.20537,46.943641,0.0,45278.0,5864.718929,3645.129568


LD morphology metrics table:


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,LD,6,"(0.3891, 0.0799, 0.0799)",3.516378,23.676707,16.601673,5,283,192,14,312,224,29.001591,8.234034,3.522161,2.505367,0.397150,1,0.827388,3.315367,1042.0,43756.0,18047.386192,8371.774996
1,LD,10,"(0.3891, 0.0799, 0.0799)",3.620494,27.140300,15.042350,8,334,184,12,346,194,5.073407,0.456757,11.107458,0.955496,0.383333,1,0.760331,1.588509,1728.0,26111.0,12033.972826,4503.025403


#### **`STEP 3` - Combine all of the tables together and add column**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block combines the above tables together so that each organelle object is listed as a separate column in a single table. A new column is then added to specify which image the data is from.

In [72]:
# combine the lists for each organelle into one table
final_org_tab = pd.concat(org_tabs, ignore_index=True)

# add a new column to list the name of the image these data are derived from 
final_org_tab.insert(loc=0,column='image_name',value=file_path.stem)

# print table for inspection
display(final_org_tab)

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,a24hrs_Ctrl_14_Unmixing,lyso,1,"(0.3891, 0.0799, 0.0799)",0.075313,21.835879,15.873847,0,271,197,2,277,202,1.001157,0.076954,13.009875,0.527728,0.516667,1,0.837838,0.745434,0.0,6740.0,2662.774194,1873.980096
1,a24hrs_Ctrl_14_Unmixing,lyso,2,"(0.3891, 0.0799, 0.0799)",0.000000,21.685153,20.527015,0,271,257,1,273,258,0.152282,0.004965,30.672726,0.211657,1.000000,1,inf,0.178598,992.0,2471.0,1731.500000,739.500000
2,a24hrs_Ctrl_14_Unmixing,lyso,3,"(0.3891, 0.0799, 0.0799)",0.000000,22.265759,19.359660,0,276,239,1,283,246,0.806408,0.064542,12.494366,0.497677,0.530612,1,inf,0.860433,174.0,4247.0,1975.115385,968.798992
3,a24hrs_Ctrl_14_Unmixing,lyso,4,"(0.3891, 0.0799, 0.0799)",3.172964,23.723579,17.715020,0,226,167,16,361,283,389.806620,65.135009,5.984595,4.991970,0.104721,-17,0.233281,10.902797,0.0,24052.0,5743.695110,3725.788256
4,a24hrs_Ctrl_14_Unmixing,lyso,5,"(0.3891, 0.0799, 0.0799)",0.148781,24.490058,25.988826,0,305,323,2,310,329,1.046691,0.084401,12.401451,0.544230,0.566667,1,0.918919,0.845667,0.0,5711.0,2080.500000,1527.815824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,a24hrs_Ctrl_14_Unmixing,perox,79,"(0.3891, 0.0799, 0.0799)",5.447659,24.014743,21.731745,14,299,271,15,303,275,0.907800,0.029788,30.474862,0.384606,0.750000,1,inf,0.389706,737.0,14096.0,7890.416667,4367.752406
506,a24hrs_Ctrl_14_Unmixing,perox,80,"(0.3891, 0.0799, 0.0799)",5.447659,36.667744,33.040240,14,458,412,15,462,416,0.907800,0.029788,30.474862,0.384606,0.750000,1,inf,0.389706,2233.0,12540.0,7533.166667,3317.146711
507,a24hrs_Ctrl_14_Unmixing,ER,1,"(0.3891, 0.0799, 0.0799)",2.602363,28.710334,22.730290,0,123,102,17,632,566,3622.111476,893.570207,4.053528,11.950242,0.089655,-124,0.205370,46.943641,0.0,45278.0,5864.718929,3645.129568
508,a24hrs_Ctrl_14_Unmixing,LD,6,"(0.3891, 0.0799, 0.0799)",3.516378,23.676707,16.601673,5,283,192,14,312,224,29.001591,8.234034,3.522161,2.505367,0.397150,1,0.827388,3.315367,1042.0,43756.0,18047.386192,8371.774996


#### **`DEFINE` - The get_organelle_morph() function**

> ***IMPORTANT**: The solidity measurement may cause an error for objects that are very small. It depends on the convex hull measurement which is computed as 0 when the object volume is very small; the solidity value is them output in the table as `inf` (infinity). The following error message will be included:*
> ```python
> UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
> ```

In [ ]:
def _get_organelle_morph(source_file: str,
                         list_obj_names: List[str],
                         list_obj_segs: List[np.ndarray],
                         list_intensity_img: List[np.ndarray],
                         list_region_names: List[str],
                         list_region_segs: List[np.ndarray],
                         mask: str,
                         scale: Union[tuple,None] = None):
    """
    Measure the composition, morphology, distribution, and contacts of multiple organelles in a cell

    Parameters:
    ----------
    source_file: str
        file path; this is used for recorder keeping of the file name in the output data tables
    list_obj_names: List[str]
        a list of object names (strings) that will be measured; this should match the order in list_obj_segs
    list_obj_segs: List[np.ndarray]
        a list of 3D (ZYX) segmentation np.ndarrays that will be measured per cell; the order should match the list_obj_names 
    list_intensity_img: List[np.ndarray]
        a list of 3D (ZYX) grayscale np.ndarrays that will be used to measure fluoresence intensity in each region and object
    list_region_names: List[str]
        a list of region names (strings); these should include the mask (entire region being measured - usually the cell) 
        and other sub-mask regions from which we can meausure the objects in (ex - nucleus, neurites, soma, etc.). It should 
        also include the centering object used when created the XY distribution bins.
        The order should match the list_region_segs
    list_region_segs: List[np.ndarray]
        a list of 3D (ZYX) binary np.ndarrays of the region masks; the order should match the list_region_names.
    mask: str
        a str of which region name (contained in the list_region_names list) should be used as the main mask (e.g., cell mask)
    scale: Union[tuple,None] = None
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)

    Returns:
    ----------
    Dataframe of measurements of organelle morphology

    """
    # select the mask from the region list
    mask = list_region_segs[list_region_names.index(mask)]
    
    # empty list to collect a morphology data for each organelle
    org_tabs = []

    # loop through the list of organelles and run the get_morphology_metrics function
    for j, target in enumerate(list_obj_names):
        # select intensity image
        org_img = list_intensity_img[j]  
        
        # select segmentation and if ER, ensure it is only one object
        if target == 'ER':
            org_obj = (list_obj_segs[j] > 0).astype(np.uint16)  
        else:
            org_obj = list_obj_segs[j]
        
        # run get_morphology_metrics function to output a table of measurements
        org_metrics = get_morphology_metrics(segmentation_img=org_obj, 
                                            seg_name=target,
                                            intensity_img=org_img, 
                                            mask=mask,
                                            scale=scale)

        # add table to list above
        org_tabs.append(org_metrics)

    # combine the lists for each organelle into one table
    final_org_tab = pd.concat(org_tabs, ignore_index=True)

    # add a new column to list the name of the image these data are derived from 
    final_org_tab.insert(loc=0,column='image_name',value=source_file.stem)

    return final_org_tab

In [58]:
org_morph_tab = _get_organelle_morph(source_file = file_path,
                                     list_obj_names = org_file_names,
                                     list_obj_segs = organelles,
                                     list_intensity_img = intensities, 
                                     list_region_names = regions_file_names,
                                     list_region_segs = regions,
                                     mask=mask_name,
                                     scale=scale)
org_morph_tab

c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6214 qhull input error: not enough points(2) to construct initial simplex (need 4)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 285131638  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:629: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,a24hrs_Ctrl_14_Unmixing,lyso,1,"(0.3891, 0.0799, 0.0799)",0.075313,21.835879,15.873847,0,271,197,2,277,202,1.001157,0.076954,13.009875,0.527728,0.516667,1,0.837838,0.745434,0.0,6740.0,2662.774194,1873.980096
1,a24hrs_Ctrl_14_Unmixing,lyso,2,"(0.3891, 0.0799, 0.0799)",0.000000,21.685153,20.527015,0,271,257,1,273,258,0.152282,0.004965,30.672726,0.211657,1.000000,1,inf,0.178598,992.0,2471.0,1731.500000,739.500000
2,a24hrs_Ctrl_14_Unmixing,lyso,3,"(0.3891, 0.0799, 0.0799)",0.000000,22.265759,19.359660,0,276,239,1,283,246,0.806408,0.064542,12.494366,0.497677,0.530612,1,inf,0.860433,174.0,4247.0,1975.115385,968.798992
3,a24hrs_Ctrl_14_Unmixing,lyso,4,"(0.3891, 0.0799, 0.0799)",3.172964,23.723579,17.715020,0,226,167,16,361,283,389.806620,65.135009,5.984595,4.991970,0.104721,-17,0.233281,10.902797,0.0,24052.0,5743.695110,3725.788256
4,a24hrs_Ctrl_14_Unmixing,lyso,5,"(0.3891, 0.0799, 0.0799)",0.148781,24.490058,25.988826,0,305,323,2,310,329,1.046691,0.084401,12.401451,0.544230,0.566667,1,0.918919,0.845667,0.0,5711.0,2080.500000,1527.815824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,a24hrs_Ctrl_14_Unmixing,perox,79,"(0.3891, 0.0799, 0.0799)",5.447659,24.014743,21.731745,14,299,271,15,303,275,0.907800,0.029788,30.474862,0.384606,0.750000,1,inf,0.389706,737.0,14096.0,7890.416667,4367.752406
506,a24hrs_Ctrl_14_Unmixing,perox,80,"(0.3891, 0.0799, 0.0799)",5.447659,36.667744,33.040240,14,458,412,15,462,416,0.907800,0.029788,30.474862,0.384606,0.750000,1,inf,0.389706,2233.0,12540.0,7533.166667,3317.146711
507,a24hrs_Ctrl_14_Unmixing,ER,1,"(0.3891, 0.0799, 0.0799)",2.602363,28.710334,22.730290,0,123,102,17,632,566,3622.111476,893.570207,4.053528,11.950242,0.089655,-124,0.205370,46.943641,0.0,45278.0,5864.718929,3645.129568
508,a24hrs_Ctrl_14_Unmixing,LD,6,"(0.3891, 0.0799, 0.0799)",3.516378,23.676707,16.601673,5,283,192,14,312,224,29.001591,8.234034,3.522161,2.505367,0.397150,1,0.827388,3.315367,1042.0,43756.0,18047.386192,8371.774996


### 🧪 **Process Multiple Cells (from a single experiment)**

In [ ]:
def _batch_process_org_morph(out_file_name: str,
                                  seg_path: Union[Path,str],
                                  out_path: Union[Path, str], 
                                  raw_path: Union[Path,str], 
                                  raw_file_type: str,
                                  organelle_names: List[str],
                                  organelle_channels: List[int],
                                  region_names: List[str],
                                  masks_file_name: str,
                                  mask: str,
                                  dist_centering_obj:str, 
                                  dist_num_bins: int,
                                  dist_center_on: bool=False,
                                  dist_keep_center_as_bin: bool=True,
                                  dist_zernike_degrees: Union[int, None]=None,
                                  include_contact_dist: bool = True,
                                  scale:bool=True,
                                  seg_suffix:Union[str, None]=None) -> int :
    """  
    batch process segmentation quantification (morphology, distribution, contacts); this function is currently optimized to process images from one file folder per image type (e.g., raw, segmentation)
    the output csv files are saved to the indicated out_path folder

    Parameters:
    ----------
    out_file_name: str
        the prefix to use when naming the output datatables
    seg_path: Union[Path,str]
        Path or str to the folder that contains the segmentation tiff files
    out_path: Union[Path, str]
        Path or str to the folder that the output datatables will be saved to
    raw_path: Union[Path,str]
        Path or str to the folder that contains the raw image files
    raw_file_type: str
        the file type of the raw data; ex - ".tiff", ".czi"
    organelle_names: List[str]
        a list of all organelle names that will be analyzed; the names should be the same as the suffix used to name each of the tiff segmentation files
        Note: the intensity measurements collect per region (from get_region_morphology_3D function) will only be from channels associated to these organelles 
    organelle_channels: List[int]
        a list of channel indices associated to respective organelle staining in the raw image; the indices should listed in same order in which the respective segmentation name is listed in organelle_names
    region_names: List[str]
        a list of regions, or masks, to measure; the order should correlate to the order of the channels in the "masks" output segmentation file
    masks_file_name: str
        the suffix of the "masks" segmentation file; ex- "masks_B", "masks", etc.
        this function currently does not accept indivial region segmentations 
    mask: str
        the name of the region to use as the mask when measuring the organelles; this should be one of the names listed in regions list; usually this will be the "cell" mask
    dist_centering_obj:str
        the name of the region or object to use as the centering object in the get_XY_distribution function
    dist_num_bins: int
        the number of bins for the get_XY_distribution function
    dist_center_on: bool=False,
        for get_XY_distribution:
        True = distribute the bins from the center of the centering object
        False = distribute the bins from the edge of the centering object
    dist_keep_center_as_bin: bool=True
        for get_XY_distribution:
        True = include the centering object area when creating the bins
        False = do not include the centering object area when creating the bins
    dist_zernike_degrees: Union[int, None]=None
        for get_XY_distribution:
        the number of zernike degrees to include for the zernike shape descriptors; if None, the zernike measurements will not 
        be included in the output
    include_contact_dist:bool=True
        whether to include the distribution of contact sites in get_contact_metrics_3d(); True = include contact distribution
    scale:bool=True
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)
    seg_suffix:Union[str, None]=None
        any additional text that is included in the segmentation tiff files between the file stem and the segmentation suffix
    


    Returns:
    ----------
    count: int
        the number of images processed
        
    """
    start = time.time()
    count = 0

    # create path objects from strings
    if isinstance(raw_path, str): raw_path = Path(raw_path)
    if isinstance(seg_path, str): seg_path = Path(seg_path)
    if isinstance(out_path, str): out_path = Path(out_path)
    
    # create directory is it doesn't exist
    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")
    

    # reading list of files from the raw path
    img_file_list = list_image_files(raw_path, raw_file_type)

    # list of organelle segmentation and masks files to collect from each image
    segs_to_collect = organelle_names + [masks_file_name]

    # containers to collect data tabels
    org_tabs = []
    contact_tabs = []
    dist_tabs = []
    region_tabs = []
    for img_f in img_file_list:
        count = count + 1
        filez = find_segmentation_tiff_files(img_f, segs_to_collect, seg_path, seg_suffix)

        # read in raw file and metadata
        img_data, meta_dict = read_czi_image(filez["raw"])

        # create intensities from raw file as list based on the channel order provided
        intensities = [img_data[ch] for ch in organelle_channels]

        # define the scale
        if scale is True:
            scale_tup = meta_dict['scale']
        else:
            scale_tup = None

        # load regions as a list based on order in list (should match order in "masks" file)
        masks = read_tiff_image(filez[masks_file_name]) 
        regions = [masks[r] for r, region in enumerate(region_names)]

        # store organelle images as list
        organelles = [read_tiff_image(filez[org]) for org in organelle_names]

        org_metrics, contact_metrics, dist_metrics, region_metrics = make_all_metrics_tables(source_file=img_f,
                                                                                             list_obj_names=organelle_names,
                                                                                             list_obj_segs=organelles,
                                                                                             list_intensity_img=intensities, 
                                                                                             list_region_names=region_names,
                                                                                             list_region_segs=regions, 
                                                                                             mask=mask,
                                                                                             dist_centering_obj=dist_centering_obj,
                                                                                             dist_num_bins=dist_num_bins,
                                                                                             dist_center_on=dist_center_on,
                                                                                             dist_keep_center_as_bin=dist_keep_center_as_bin,
                                                                                             dist_zernike_degrees=dist_zernike_degrees,
                                                                                             scale=scale_tup,
                                                                                             include_contact_dist=include_contact_dist)

        org_tabs.append(org_metrics)
        contact_tabs.append(contact_metrics)
        dist_tabs.append(dist_metrics)
        region_tabs.append(region_metrics)
        end2 = time.time()
        print(f"Completed processing for {count} images in {(end2-start)/60} mins.")

    final_org = pd.concat(org_tabs, ignore_index=True)
    final_contact = pd.concat(contact_tabs, ignore_index=True)
    final_dist = pd.concat(dist_tabs, ignore_index=True)
    final_region = pd.concat(region_tabs, ignore_index=True)

    org_csv_path = out_path / f"{out_file_name}_organelles.csv"
    final_org.to_csv(org_csv_path)

    contact_csv_path = out_path / f"{out_file_name}_contacts.csv"
    final_contact.to_csv(contact_csv_path)

    dist_csv_path = out_path / f"{out_file_name}_distributions.csv"
    final_dist.to_csv(dist_csv_path)

    region_csv_path = out_path / f"{out_file_name}_regions.csv"
    final_region.to_csv(region_csv_path)

    end = time.time()
    print(f"Quantification for {count} files is COMPLETE! Files saved to '{out_path}'.")
    print(f"It took {(end - start)/60} minutes to quantify these files.")
    return count

### 🧮 **Summarize Metrics per Cell Across One or More Experiments**

-----

## **QUANTIFICATION**